# Import libraries

In [1]:
from pathlib import Path
from config.load_config import load_config

# Create enviroment variables

In [4]:
cfg = load_config()

In [5]:
BASE_DIR = Path(cfg["base_dir"]).resolve()
print(BASE_DIR)
OUTPUT_DIR = BASE_DIR / cfg["output_subdir"]
print(OUTPUT_DIR)

/home/julian/Documents/Julian/ML_Engineer/repos/ml-object-detector
/home/julian/Documents/Julian/ML_Engineer/repos/ml-object-detector/output_images


# Create folder to output images

In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)